In [1]:
import sys
sys.path.append('../src')
from dataloader import create_dataloaders
# from dataloader_ER import create_dataloaders
# from lora_model import LORAEngine,LORAEngineDeberta,LORAEngineDebertaMultiClass
from lora_model_vmap import LORAEngineDebertaMultiClass

# from influence import IFEngine
from influence_hyperinf import IFEngine

import numpy as np
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
model_name_or_path="../../model/deberta-v3-base"
# model_name_or_path="../../model/roberta-large"
task="mrpc"
# task = "qnli"
noise_ratio=0
batch_size=32
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
# target_modules = ['value']
target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:0"
num_epochs=3
lr=3e-3

In [3]:
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
# model_name_or_path="../../model/deberta-v3-base"
model_name_or_path = '/data/home/wangys/model/bert-base-uncased/'
# model_name_or_path="../../model/roberta-large"
task="mrpc"
# task = "qnli"
noise_ratio=0
batch_size=32
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
target_modules = ['value']
# target_modules=["value_proj"] ## deberta, for roberta, use ["value"]

device="cuda:0"
num_epochs=3
lr=3e-3

In [4]:
# fine-tuning models
# dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
#                                            task=task,
#                                            noise_ratio=noise_ratio,
#                                            batch_size=batch_size,
#                                            train_file = '../ER/semi-text-c-FUSER/train.json',
#                                            valid_file = '../ER/semi-text-c-FUSER/valid.json',
#                                            test_file = '../ER/semi-text-c-FUSER/test.json')


dataloader_outputs = create_dataloaders(model_name_or_path=model_name_or_path,
                                           task=task,
                                           noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                        #    train_file = '../ER/semi-text-c/train.json',
                                        #    valid_file = '../ER/semi-text-c/valid.json',
                                        #    test_file = '../ER/semi-text-c/test.json',
                                           max_length=256)
# train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs
train_dataloader, eval_dataloader,test_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs


# lora_engine = LORAEngine(model_name_or_path=model_name_or_path,
#                             target_modules=target_modules,
#                             train_dataloader=train_dataloader,
#                             eval_dataloader=eval_dataloader,
#                             device=device,
#                             num_epochs=num_epochs,
#                             lr=lr,
#                             low_rank=8, 
#                             task=task)

lora_engine = LORAEngineDebertaMultiClass(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/mrpc',
                            valid_each_epoch=True)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [6]:
for step,batch in enumerate(train_dataloader):
    print(batch['id'])
    break
    # print(batch['input_ids'][0])
    

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([2695, 1713, 1224, 1780,  153, 1512, 2824, 2531,  204, 3084, 2186, 2199,
        2536, 1038,  696, 1407,  398,  862, 2327, 1679, 3104, 3391, 1940, 1613,
         108, 1194,  684, 3215, 1552, 2827, 2723, 1935])


In [5]:
# 
lora_engine.eval_LORA_model()

AttributeError: 'LORAEngineDebertaMultiClass' object has no attribute 'model'

In [6]:
lora_engine.build_LORA_model()
lora_engine.train_LORA_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/home/wangys/model/bert-base-uncased/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700
Total Steps:345


100%|██████████| 115/115 [00:06<00:00, 17.14it/s]


Epoch 1: Training Loss = 0.5690182263436525


100%|██████████| 13/13 [00:00<00:00, 27.58it/s]


...

Epoch 3: Training Loss = 0.2843996771003889


100%|██████████| 13/13 [00:00<00:00, 28.07it/s]


Epoch 3: {'f1': 0.9}


100%|██████████| 54/54 [00:01<00:00, 30.20it/s]

Prediction Result on Test Data: {'f1': 0.8831385642737897}


In [6]:
lora_engine.save('../models/mrpc-all')
# lora_engine.load('../models/qnli')

[2024-12-16 17:24:04,892] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/wangys/anaconda3/envs/deepspeed/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


In [7]:
lora_engine.save_lora('../models/mrpc-bert-lora')


In [10]:
lora_engine.load_pretrained_network('../models/mrpc')
lora_engine.eval_LORA_model()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at ../../model/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 54/54 [00:02<00:00, 22.75it/s]

Prediction Result on Test Data: {'f1': 0.7987465181058496}


In [12]:
tr_grad_dict, val_grad_dict = lora_engine.compute_gradient_iterative(tokenized_datasets, collate_fn, batch_size=32)

100%|██████████| 13/13 [00:01<00:00, 11.82it/s]


In [18]:
tr_grad_dict[9]['ids']

tensor([288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301,
        302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315,
        316, 317, 318, 319])

In [18]:
for step,batch in enumerate(test_dataloader):
    print(batch)
    break

{'labels': tensor([1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 0]), 'input_ids': tensor([[    1,  2732, 28702,  ...,     2,     2,     2],
        [    1,   279,   447,  ..., 18905,   323,     2],
        [    1,  1663,   264,  ...,     2,     2,     2],
        ...,
        [    1, 20415, 99165,  ...,     2,     2,     2],
        [    1, 90792,   268,  ...,     2,     2,     2],
        [    1,   279,  1323,  ...,     2,     2,     2]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [16]:
val_grad_dict[0]['base_model.model.deberta.encoder.layer.1.attention.self.value_proj.lora_B.default.weight'].shape

torch.Size([16, 768])